In [ ]:
import pandas as pd
import pickle
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [ ]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [ ]:
# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/Mobile Prioritazitation/Final_dataset.csv')
df.info()

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
df['sentiment'] = labelencoder.fit_transform(df['sentiment'])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sentiment_categories = [1, 2, 3, 4, 5]

ax=sns.countplot(x='sentiment', data=df)
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.0f'),
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha = 'center', va = 'center',
                xytext = (0, 6),
                textcoords = 'offset points')
# Setting x-tick labels to start from 1
ax.set_xticklabels(sentiment_categories)
# Adding x and y titles
plt.xlabel('Priority')
plt.ylabel('No. of Reviews')

plt.show()

In [ ]:
# Assuming df is your DataFrame and 'column1' and 'column2' are the column names you want to join
df['review'] = df['title'].fillna('') + ' ' + df['body'].fillna('')

In [ ]:
def preprocess_reviews(df, review_column, preprocessed_column):
    # Convert text to lowercase
    df[preprocessed_column] = df[review_column].str.lower()

    # Replace NaN values with an empty string
    df[preprocessed_column].fillna('', inplace=True)

    # Remove numbers
    df[preprocessed_column] = df[preprocessed_column].apply(lambda x: re.sub(r'\d+', '', x))

    # Remove punctuation
    df[preprocessed_column] = df[preprocessed_column].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

    # Tokenize text
    df[preprocessed_column] = df[preprocessed_column].apply(word_tokenize)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    df[preprocessed_column] = df[preprocessed_column].apply(lambda x: [word for word in x if word not in stop_words])

    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    df[preprocessed_column] = df[preprocessed_column].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

    # Join tokens back into a string
    df[preprocessed_column] = df[preprocessed_column].apply(lambda x: ' '.join(x))

    return df

# Assuming 'df' is your DataFrame
df = preprocess_reviews(df, 'review', 'preprocessed')

In [ ]:
import pandas as pd
# Save the updated DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Mobile Prioritazitation/preprocessed_df.csv', index=False)


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Mobile Prioritazitation/preprocessed_df.csv')

In [ ]:
df

In [ ]:
# # Preprocess the text data
# text_data = df['preprocessed']  # Replace 'text_column' with the name of the column containing the text data
# # text_data = df['review']  # Replace 'text_column' with the name of the column containing the text data
# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(text_data)

# # Split the dataset into features (X) and labels (y)
# y = df['star']  # Replace 'target_column' with the actual name of your target column

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Adjust the test_size and random_state as desired

# # Create an SVM classifier
# classifier = SVC(kernel='linear')

# # Train the classifier on the training data
# classifier.fit(X_train, y_train)

# # Make predictions on the test data
# y_pred = classifier.predict(X_test)

# # Evaluate the accuracy of the classifier
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy}")


In [ ]:
# # Print classification report and confusion matrix
# print(classification_report(y_test, y_pred))

# # Calculate and print accuracy, precision, recall, and f1-score
# accuracy = accuracy_score(y_test, y_pred)
# precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
# print("Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-score: {:.4f}".format(accuracy, precision, recall, f1_score))


# M/DL Models using Bert Embeddings

In [ ]:
pip install transformers

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:

from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader

In [ ]:
# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
# Define dataset class
class MyDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        return text

In [ ]:
def my_collate_fn(batch):
    return [' '.join(sample) for sample in batch]

In [ ]:
# Define batch size and create data loader
batch_size = 64
dataset = MyDataset(df['review'])
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=4, collate_fn=my_collate_fn)


# Generate embeddings for each batch
embeddings = []
model.eval()
i = 1
with torch.no_grad():
    for batch in dataloader:
        # Tokenize batch
        inputs = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Generate embeddings
        print("batch ")
        outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        mean_pooling = torch.mean(last_hidden_states, dim=1)
        embeddings.append(mean_pooling.cpu())

# Concatenate embeddings from each batch
embeddings = torch.cat(embeddings, dim=0)

In [ ]:
import pickle
with open("/content/drive/MyDrive/Mobile Prioritazitation/MP_df_embeddings_.pickle", "wb") as scores:
    pickle.dump(embeddings, scores)

In [ ]:
import pickle

with open("/content/drive/MyDrive/Mobile Prioritazitation/MP_df_embeddings_.pickle", "rb") as scores:
   embeddings = pickle.load(scores)

In [ ]:
embeddings.shape

In [ ]:
from sklearn.svm import SVC

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(embeddings, df['sentiment'], test_size=0.20, random_state=42)

# Create an SVM model and fit it to the training data
svm_model = SVC()
svm_model.fit(X_train, y_train)

# Predict on test data
y_pred = svm_model.predict(X_test)

# Print classification report and confusion matrix
print(classification_report(y_test, y_pred))

# Multinomial Naive Bays

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(embeddings, df['sentiment'], test_size=0.20, random_state=42)

# Binning the embeddings
threshold = 0.5
X_train_bin = np.where(X_train > threshold, 1, 0)
X_test_bin = np.where(X_test > threshold, 1, 0)

# Train the MNB model
mnb = MultinomialNB()
mnb.fit(X_train_bin, y_train)

# Predict on test data
y_pred = mnb.predict(X_test_bin)

# Print classification report and accuracy score
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

# Calculate and print accuracy, precision, recall, and f1-score
accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
print("Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-score: {:.4f}".format(accuracy, precision, recall, f1_score))

In [ ]:
# Calculate and print accuracy, precision, recall, and f1-score
accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
print("Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-score: {:.4f}".format(accuracy, precision, recall, f1_score))

# Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(embeddings, df['sentiment'], test_size=0.20, random_state=42)

# Create a Logistic Regression model and fit it to the training data
logreg_model = LogisticRegression()
logreg_model.fit(X_train, y_train)

# Predict on test data
y_pred = logreg_model.predict(X_test)

# Print classification report and confusion matrix
print(classification_report(y_test, y_pred))

In [ ]:
# Calculate and print accuracy, precision, recall, and f1-score
accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
print("Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-score: {:.4f}".format(accuracy, precision, recall, f1_score))

# Random forest

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(embeddings, df['sentiment'], test_size=0.20, random_state=42)

# Create a Random Forest model and fit it to the training data
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict on test data
y_pred = rf_model.predict(X_test)

# Print classification report and confusion matrix
print(classification_report(y_test, y_pred))

# Calculate and print accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate and print accuracy, precision, recall, and f1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
print("Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-score: {:.4f}".format(accuracy, precision, recall, f1_score))

In [ ]:
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import torch
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

# GradientBoostingClassifier

In [ ]:
# # Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(embeddings, df['sentiment'], test_size=0.20, random_state=42)

# Define the boosting ensemble model
gb_model = GradientBoostingClassifier(n_estimators=10, learning_rate=0.1, max_depth=3, random_state=42)

# Train the model on the training set
gb_model.fit(X_train, y_train)

y_pred_prob_gb = gb_model.predict_proba(X_test.reshape((X_test.shape[0], -1)))
y_pred_gb = np.round(y_pred_prob_gb[:, 1]).astype('int')


# Print the classification report
print(classification_report(y_test, y_pred_gb))

In [ ]:
# Calculate and print accuracy, precision, recall, and f1-score
print("GBM.....")
accuracy = accuracy_score(y_test, y_pred_gb)
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred_gb, average='weighted')
print("Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-score: {:.4f}".format(accuracy, precision, recall, f1_score))

In [ ]:
# Calculate count of each label
unique_labels, label_counts = np.unique(df['star'], return_counts=True)

# Print results
for label, count in zip(unique_labels, label_counts):
    print(f"Label {label}: {count} instances")

# LSTM on Ramdom Splition

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(embeddings.numpy(), df['sentiment'].values, test_size=0.20, random_state=42)


# Reshape the input data
X_train = X_train[:, np.newaxis, :]
X_test = X_test[:, np.newaxis, :]

# Convert numpy arrays to PyTorch tensors
X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

# Set the parameters for the LSTM model
input_size = X_train.shape[2]  # Number of features in the input (embedding size)
hidden_size = 128  # Number of LSTM units
output_size = 5  # Number of classes (adjust this based on your actual number of classes)



# ... (previous code) ...

# Define the LSTM model with an additional LSTM layer and dropout
class ImprovedLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ImprovedLSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm1(x, (h0, c0))
        out, _ = self.lstm2(out)
        out = self.dropout(out)
        out = self.fc(out[:, -1, :])
        return out

# Create an instance of the Improved LSTM model
improved_model = ImprovedLSTMModel(input_size, hidden_size, output_size)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(improved_model.parameters(), lr=0.001)

# Create data loaders
train_dataset = TensorDataset(X_train, y_train.long())
test_dataset = TensorDataset(X_test, y_test.long())
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# Train the model
epochs = 10
for epoch in range(epochs):
    improved_model.train()
    total_loss = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = improved_model(inputs.float())
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader)}")

# Evaluate the improved model on the test set
improved_model.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = improved_model(inputs.float())
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.tolist())
        true_labels.extend(targets.tolist())

# Convert predictions and true labels to numpy arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Calculate accuracy, precision, recall, and F1-score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = 100 * accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='macro')
recall = recall_score(true_labels, predictions, average='macro')
f1 = f1_score(true_labels, predictions, average='macro')

print(f"Test Accuracy: {accuracy:.4f}%")
print(f"Precision: {precision:.5f}")
print(f"Recall: {recall:.5f}")
print(f"F1-score: {f1:.5f}")

In [ ]:
# Calculate precision, recall, and F1-score
from sklearn.metrics import precision_score, recall_score, f1_score
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='macro')
recall = recall_score(true_labels, predictions, average='macro')
f1 = f1_score(true_labels, predictions, average='macro')
print(f"accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# Simple Neural Network Model

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(embeddings.numpy(), df['sentiment'].values, test_size=0.20, random_state=42)


In [ ]:
# Convert numpy arrays to PyTorch tensors
X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

# Define the Neural Network model
class SimpleNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Set the parameters for the neural network model
input_size = X_train.shape[1]  # Number of features in the input (embedding size)
hidden_size = 128  # Number of units in the hidden layer
output_size = 10  # Number of classes (adjust this based on your actual number of classes)

# Create an instance of the Neural Network model
model = SimpleNNModel(input_size, hidden_size, output_size)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create data loaders
train_dataset = TensorDataset(X_train, y_train.long())
test_dataset = TensorDataset(X_test, y_test.long())
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# Train the model
epochs = 10
learning_rate = 0.001
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader)}")

# Evaluate the model on the test set
model.eval()
predictions = []
true_labels = []
correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.tolist())
        true_labels.extend(targets.tolist())
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

# Convert predictions and true labels to numpy arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Calculate accuracy, precision, recall, and F1-score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = 100 * correct / total
precision = precision_score(true_labels, predictions, average='macro')
recall = recall_score(true_labels, predictions, average='macro')
f1 = f1_score(true_labels, predictions, average='macro')

print(f"Test Accuracy: {accuracy:.2f}%")
print(f"Precision: {precision:.5f}")
print(f"Recall: {recall:.5f}")
print(f"F1-score: {f1:.5f}")

# Simle CNN

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(embeddings.numpy(), df['sentiment'].values, test_size=0.20, random_state=42)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# Assuming you have already imported the required libraries and created 'X_train', 'X_test', 'y_train', and 'y_test'.

# Convert numpy arrays to PyTorch tensors
X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

# Define the CNN model
class AdvancedCNNModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(AdvancedCNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(256 * (X_train.shape[-1] // 8), 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.conv2(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.conv3(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Set the parameters for the CNN model
input_channels = 1  # Number of channels in the input data (since it's one-dimensional)
num_classes = len(set(y_train.tolist()))  # Number of classes (adjust based on your actual number of classes)

# Create an instance of the Advanced CNN model
model = AdvancedCNNModel(input_channels, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create data loaders
train_dataset = TensorDataset(X_train.unsqueeze(1), y_train.long())  # Add an extra dimension for the channel
test_dataset = TensorDataset(X_test.unsqueeze(1), y_test.long())
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# Train the model
epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader)}")

# Evaluate the model on the test set
model.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.tolist())
        true_labels.extend(targets.tolist())

# Convert predictions and true labels to numpy arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Calculate accuracy, precision, recall, and F1-score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = 100 * accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='macro')
recall = recall_score(true_labels, predictions, average='macro')
f1 = f1_score(true_labels, predictions, average='macro')

print(f"Test Accuracy: {accuracy:.2f}%")
print(f"Precision: {precision:.5f}")
print(f"Recall: {recall:.5f}")
print(f"F1-score: {f1:.5f}")


# XGBClassifier

In [ ]:
pip install xgboost

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Split data into training and test sets (assuming 'embeddings' is a NumPy array)
X_train, X_test, y_train, y_test = train_test_split(embeddings, df['sentiment'], test_size=0.20, random_state=42)

# Create the XGBoost model
model = xgb.XGBClassifier(
    objective='multi:softmax',  # Multiclass classification
    num_class=len(set(y_train)),  # Number of classes
    learning_rate=0.1,
    n_estimators=100,  # Number of trees (boosting rounds)
    max_depth=5,  # Maximum tree depth
    subsample=0.8,  # Subsample ratio of the training instances
    colsample_bytree=0.8,  # Subsample ratio of features for building each tree
    random_state=42
)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics
accuracy = 100 * accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print(f"Test Accuracy: {accuracy:.2f}%")
print(f"Precision: {precision:.5f}")
print(f"Recall: {recall:.5f}")
print(f"F1-score: {f1:.5f}")

# DT

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Split data into training and test sets (assuming 'embeddings' is a NumPy array)
X_train, X_test, y_train, y_test = train_test_split(embeddings, df['sentiment'], test_size=0.20, random_state=42)
# Create a Decision Tree classifier
clf = DecisionTreeClassifier(random_state=42)

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
predictions = clf.predict(X_test)

# Calculate accuracy, precision, recall, and F1-score
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# RNN

In [ ]:
embeddings.shape

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# Assuming 'embeddings' is a torch.Tensor 
X_train, X_test, y_train, y_test = train_test_split(embeddings.numpy(), df['sentiment'], test_size=0.20, random_state=42)

# Convert numpy arrays to PyTorch tensors using the recommended approach
X_train = torch.tensor(X_train).clone().detach()
X_test = torch.tensor(X_test).clone().detach()
y_train = torch.tensor(y_train.values).clone().detach()  # Convert to tensor and use .values to get the underlying numpy array
y_test = torch.tensor(y_test.values).clone().detach()  # Convert to tensor and use .values to get the underlying numpy array

# Set the parameters for the RNN model
input_size = X_train.shape[1]  # Number of features in the input (embedding size)
hidden_size = 128  # Number of units in the hidden layer
output_size = len(set(y_train.tolist()))  # Number of classes (adjust based on your actual number of classes)

# Define the RNN model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x[:, np.newaxis, :], h0)  # Add a new dimension to the input tensor
        out = self.fc(out[:, -1, :])  # Take the last time step's output for classification
        return out

# Create an instance of the RNN model
rnn_model = RNNModel(input_size, hidden_size, output_size)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn_model.parameters(), lr=0.001)

# Create data loaders
train_dataset = TensorDataset(X_train, y_train.long())
test_dataset = TensorDataset(X_test, y_test.long())
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# Train the model
epochs = 20
for epoch in range(epochs):
    rnn_model.train()
    total_loss = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = rnn_model(inputs.float())
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader)}")

# Evaluate the RNN model on the test set
rnn_model.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = rnn_model(inputs.float())
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.tolist())
        true_labels.extend(targets.tolist())

# Convert predictions and true labels to numpy arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Calculate accuracy, precision, recall, and F1-score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = 100 * accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='macro')
recall = recall_score(true_labels, predictions, average='macro')
f1 = f1_score(true_labels, predictions, average='macro')

print(f"Test Accuracy: {accuracy:.4f}%")
print(f"Precision: {precision:.5f}")
print(f"Recall: {recall:.5f}")
print(f"F1-score: {f1:.5f}")

# Adaboost

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming 'embeddings' is a torch.Tensor
X_train, X_test, y_train, y_test = train_test_split(embeddings.numpy(), df['sentiment'], test_size=0.20, random_state=42)

# Convert y_train and y_test to numpy arrays
y_train = y_train.values
y_test = y_test.values

# Create an instance of DecisionTreeClassifier as the base estimator
base_estimator = DecisionTreeClassifier(max_depth=2)

# Create the AdaBoostClassifier with the base estimator
adaboost_model = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=50, random_state=42)

# Train the AdaBoost model
adaboost_model.fit(X_train, y_train)

# Predict on the test set
predictions = adaboost_model.predict(X_test)

# Calculate accuracy, precision, recall, and F1-score
accuracy = 100 * accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')

print(f"Test Accuracy: {accuracy:.4f}%")
print(f"Precision: {precision:.5f}")
print(f"Recall: {recall:.5f}")
print(f"F1-score: {f1:.5f}")


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
# XLNET 

from transformers import XLNetForSequenceClassification, XLNetTokenizer

# Prepare data from the loaded df
texts = df['review'].tolist()
labels = df['sentiment'].tolist()

print(f"Total samples: {len(texts)}")

# Split data (using same split as BERT for fair comparison)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

print(f"Training samples: {len(train_texts)}")
print(f"Validation samples: {len(val_texts)}")

# Tokenize with XLNet
xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
print("\nTokenizing text data with XLNet...")
train_encodings = xlnet_tokenizer(train_texts, truncation=True, padding=True, max_length=256)
val_encodings = xlnet_tokenizer(val_texts, truncation=True, padding=True, max_length=256)

# Create datasets
train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

# Create dataloaders
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

print(f"\nBatch size: {batch_size}")
print(f"Training batches: {len(train_dataloader)}")

# Load XLNet model
num_labels = len(set(labels))
xlnet_model = XLNetForSequenceClassification.from_pretrained(
    'xlnet-base-cased',
    num_labels=num_labels,
    output_attentions=False,
    output_hidden_states=False
)
xlnet_model.to(device)

print(f"\nModel loaded: XLNet-Base")
print(f"Number of classes: {num_labels}")

# Set up optimizer and scheduler
epochs = 16
optimizer = AdamW(xlnet_model.parameters(), lr=2e-5, eps=1e-8)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

print(f"\nTraining setup:")
print(f"Epochs: {epochs}")
print(f"Learning rate: {2e-5}")

# Training loop
print(f"\n{'='*60}")
print("STARTING XLNET TRAINING")
print('='*60)

xlnet_losses = []
xlnet_accuracies = []

start_time = time.time()
xlnet_model.train()

for epoch in range(epochs):
    epoch_loss = 0
    epoch_correct = 0
    total_batches = len(train_dataloader)
    
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        xlnet_model.zero_grad()
        outputs = xlnet_model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        epoch_loss += loss.item()
        epoch_correct += torch.sum(torch.argmax(logits, dim=1) == labels)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(xlnet_model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    # Calculate epoch metrics
    avg_loss = epoch_loss / total_batches
    avg_acc = epoch_correct.float() / (total_batches * batch_size)
    
    xlnet_losses.append(avg_loss)
    xlnet_accuracies.append(avg_acc.item())
    
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

xlnet_training_time = time.time() - start_time
print(f"\nXLNet Training Time: {xlnet_training_time:.2f} seconds")

# Evaluation
print(f"\n{'='*60}")
print("EVALUATING XLNET MODEL")
print('='*60)

xlnet_model.eval()
xlnet_predictions = []
xlnet_true_labels = []

with torch.no_grad():
    for batch in val_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = xlnet_model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        
        xlnet_predictions.extend(preds.cpu().numpy())
        xlnet_true_labels.extend(labels.cpu().numpy())

# Calculate all metrics
xlnet_predictions = np.array(xlnet_predictions)
xlnet_true_labels = np.array(xlnet_true_labels)

print(f"\nEvaluation on {len(xlnet_true_labels)} samples")

# Weighted metrics
xlnet_weighted_precision = precision_score(xlnet_true_labels, xlnet_predictions, average='weighted')
xlnet_weighted_recall = recall_score(xlnet_true_labels, xlnet_predictions, average='weighted')
xlnet_weighted_f1 = f1_score(xlnet_true_labels, xlnet_predictions, average='weighted')

# Macro metrics
xlnet_macro_precision = precision_score(xlnet_true_labels, xlnet_predictions, average='macro')
xlnet_macro_recall = recall_score(xlnet_true_labels, xlnet_predictions, average='macro')
xlnet_macro_f1 = f1_score(xlnet_true_labels, xlnet_predictions, average='macro')

# Micro metrics
xlnet_micro_precision = precision_score(xlnet_true_labels, xlnet_predictions, average='micro')
xlnet_micro_recall = recall_score(xlnet_true_labels, xlnet_predictions, average='micro')
xlnet_micro_f1 = f1_score(xlnet_true_labels, xlnet_predictions, average='micro')

xlnet_accuracy = accuracy_score(xlnet_true_labels, xlnet_predictions)

print("\n" + "="*60)
print("XLNET FINE-TUNING RESULTS (16 EPOCHS)")
print("="*60)
print(f"Dataset: preprocessed_df.csv")
print(f"\nOverall Accuracy: {xlnet_accuracy:.4f}")

print("\nWeighted Averages:")
print(f"  Precision: {xlnet_weighted_precision:.4f}")
print(f"  Recall:    {xlnet_weighted_recall:.4f}")
print(f"  F1-Score:  {xlnet_weighted_f1:.4f}")

print("\nMacro Averages:")
print(f"  Precision: {xlnet_macro_precision:.4f}")
print(f"  Recall:    {xlnet_macro_recall:.4f}")
print(f"  F1-Score:  {xlnet_macro_f1:.4f}")

print("\nMicro Averages:")
print(f"  Precision: {xlnet_micro_precision:.4f}")
print(f"  Recall:    {xlnet_micro_recall:.4f}")
print(f"  F1-Score:  {xlnet_micro_f1:.4f}")

# Confusion matrix
xlnet_confusion = confusion_matrix(xlnet_true_labels, xlnet_predictions)
print(f"\nConfusion Matrix:\n{xlnet_confusion}")

# Save XLNet results
xlnet_results = {
    'accuracy': xlnet_accuracy,
    'weighted': {'precision': xlnet_weighted_precision, 'recall': xlnet_weighted_recall, 'f1': xlnet_weighted_f1},
    'macro': {'precision': xlnet_macro_precision, 'recall': xlnet_macro_recall, 'f1': xlnet_macro_f1},
    'micro': {'precision': xlnet_micro_precision, 'recall': xlnet_micro_recall, 'f1': xlnet_micro_f1},
    'confusion_matrix': xlnet_confusion,
    'training_time': xlnet_training_time,
    'epochs': epochs,
    'losses': xlnet_losses,
    'accuracies': xlnet_accuracies,
    'dataset_info': {
        'source': 'preprocessed_df.csv',
        'total_samples': len(texts),
        'train_samples': len(train_texts),
        'val_samples': len(val_texts),
        'num_classes': num_labels
    }
}

In [ ]:
# ROBERTA 

from transformers import RobertaForSequenceClassification, RobertaTokenizer

# Prepare data from the loaded df
texts = df['review'].tolist()
labels = df['sentiment'].tolist()

print(f"Total samples: {len(texts)}")

# Split data (using same split for consistency)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

print(f"Training samples: {len(train_texts)}")
print(f"Validation samples: {len(val_texts)}")

# Tokenize with RoBERTa
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
print("\nTokenizing text data with RoBERTa...")
train_encodings = roberta_tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = roberta_tokenizer(val_texts, truncation=True, padding=True, max_length=128)

# Create datasets
train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

# Create dataloaders
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

print(f"\nBatch size: {batch_size}")
print(f"Training batches: {len(train_dataloader)}")

# Load RoBERTa model
num_labels = len(set(labels))
roberta_model = RobertaForSequenceClassification.from_pretrained(
    'roberta-base',
    num_labels=num_labels,
    output_attentions=False,
    output_hidden_states=False
)
roberta_model.to(device)

print(f"\nModel loaded: RoBERTa-Base")
print(f"Number of classes: {num_labels}")

# Set up optimizer and scheduler
epochs = 16
optimizer = AdamW(roberta_model.parameters(), lr=2e-5, eps=1e-8)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

print(f"\nTraining setup:")
print(f"Epochs: {epochs}")
print(f"Learning rate: {2e-5}")

# Training loop
print(f"\n{'='*60}")
print("STARTING ROBERTA TRAINING")
print('='*60)

roberta_losses = []
roberta_accuracies = []

start_time = time.time()
roberta_model.train()

for epoch in range(epochs):
    epoch_loss = 0
    epoch_correct = 0
    total_batches = len(train_dataloader)
    
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        roberta_model.zero_grad()
        outputs = roberta_model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        epoch_loss += loss.item()
        epoch_correct += torch.sum(torch.argmax(logits, dim=1) == labels)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(roberta_model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    # Calculate epoch metrics
    avg_loss = epoch_loss / total_batches
    avg_acc = epoch_correct.float() / (total_batches * batch_size)
    
    roberta_losses.append(avg_loss)
    roberta_accuracies.append(avg_acc.item())
    
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

roberta_training_time = time.time() - start_time
print(f"\nRoBERTa Training Time: {roberta_training_time:.2f} seconds")

# Evaluation
print(f"\n{'='*60}")
print("EVALUATING ROBERTA MODEL")
print('='*60)

roberta_model.eval()
roberta_predictions = []
roberta_true_labels = []

with torch.no_grad():
    for batch in val_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = roberta_model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        
        roberta_predictions.extend(preds.cpu().numpy())
        roberta_true_labels.extend(labels.cpu().numpy())

# Calculate all metrics
roberta_predictions = np.array(roberta_predictions)
roberta_true_labels = np.array(roberta_true_labels)

print(f"\nEvaluation on {len(roberta_true_labels)} samples")

# Weighted metrics
roberta_weighted_precision = precision_score(roberta_true_labels, roberta_predictions, average='weighted')
roberta_weighted_recall = recall_score(roberta_true_labels, roberta_predictions, average='weighted')
roberta_weighted_f1 = f1_score(roberta_true_labels, roberta_predictions, average='weighted')

# Macro metrics
roberta_macro_precision = precision_score(roberta_true_labels, roberta_predictions, average='macro')
roberta_macro_recall = recall_score(roberta_true_labels, roberta_predictions, average='macro')
roberta_macro_f1 = f1_score(roberta_true_labels, roberta_predictions, average='macro')

# Micro metrics
roberta_micro_precision = precision_score(roberta_true_labels, roberta_predictions, average='micro')
roberta_micro_recall = recall_score(roberta_true_labels, roberta_predictions, average='micro')
roberta_micro_f1 = f1_score(roberta_true_labels, roberta_predictions, average='micro')

roberta_accuracy = accuracy_score(roberta_true_labels, roberta_predictions)

print("\n" + "="*60)
print("ROBERTA FINE-TUNING RESULTS (16 EPOCHS)")
print("="*60)
print(f"Dataset: preprocessed_df.csv")
print(f"\nOverall Accuracy: {roberta_accuracy:.4f}")

print("\nWeighted Averages:")
print(f"  Precision: {roberta_weighted_precision:.4f}")
print(f"  Recall:    {roberta_weighted_recall:.4f}")
print(f"  F1-Score:  {roberta_weighted_f1:.4f}")

print("\nMacro Averages:")
print(f"  Precision: {roberta_macro_precision:.4f}")
print(f"  Recall:    {roberta_macro_recall:.4f}")
print(f"  F1-Score:  {roberta_macro_f1:.4f}")

print("\nMicro Averages:")
print(f"  Precision: {roberta_micro_precision:.4f}")
print(f"  Recall:    {roberta_micro_recall:.4f}")
print(f"  F1-Score:  {roberta_micro_f1:.4f}")

# Confusion matrix
roberta_confusion = confusion_matrix(roberta_true_labels, roberta_predictions)
print(f"\nConfusion Matrix:\n{roberta_confusion}")

# Save RoBERTa results
roberta_results = {
    'accuracy': roberta_accuracy,
    'weighted': {'precision': roberta_weighted_precision, 'recall': roberta_weighted_recall, 'f1': roberta_weighted_f1},
    'macro': {'precision': roberta_macro_precision, 'recall': roberta_macro_recall, 'f1': roberta_macro_f1},
    'micro': {'precision': roberta_micro_precision, 'recall': roberta_micro_recall, 'f1': roberta_micro_f1},
    'confusion_matrix': roberta_confusion,
    'training_time': roberta_training_time,
    'epochs': epochs,
    'losses': roberta_losses,
    'accuracies': roberta_accuracies,
    'dataset_info': {
        'source': 'preprocessed_df.csv',
        'total_samples': len(texts),
        'train_samples': len(train_texts),
        'val_samples': len(val_texts),
        'num_classes': num_labels
    }
}